# Gold Layer 🥇
This notebook creates the Gold Layer for the Mario Medallion Architecture. It summarizes and aggregates the cleaned data from the Silver Layer to produce analytics-ready tables.

## Imports ⬇️
Import the necessary libraries for data processing.

In [24]:
import pandas as pd
import numpy as np
import json
import numpy as np

In [25]:
def convert_for_json(obj):
    if isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    if isinstance(obj, (np.ndarray,)):
        return obj.tolist()
    return obj

def get_metrics(df):
    metrics = {}
    metrics['rows'] = int(df.shape[0])
    metrics['columns'] = int(df.shape[1])
    metrics['unique_counts'] = {col: int(df[col].nunique()) for col in df.columns}
    metrics['sums'] = {col: float(df[col].sum()) for col in df.select_dtypes(include=[np.number]).columns}
    return metrics

## Load Silver Layer Data 🔄
Load the cleaned and validated data from the Silver Layer parquet file.

In [2]:
# Load the Silver Layer parquet file
df = pd.read_parquet('../Medallion Architecture/silver/silver_transactions.parquet')

In [3]:
df.head()

,Player Name,Team,World,Vehicle Type,Companion,Kart Racing Rank,Platforming Rank,Boss Battle Rank,Power-Ups Used,Kart Role,Team Points,Lives Lost,Participation in Battle Mode,Mushroom Cup Participation,Power-Ups Owned,Coins Spent in Toad Town,Levels Completed,Times Hit by Enemies,Primary Game,mario_sim
0,Yoshi,Toad Brigade,Yoshi's Island,Comet Bike,Polterpup,A,A,A,12,Drifter,-34,0,True,True,1-Up Mushroom,64,26,4,Mario Tennis Aces,0.250000
1,Peach,Green Caps,Donut Plains,Circuit Special,Koopa Troopa,C,A,B,16,Drifter,149,4,True,True,"Red Shell, Super Star",335,40,5,Mario Tennis Aces,0.100000
2,Waluigi,No Team,Yoshi's Island,Biddybuggy,Goomba,D,A,C,26,Blocker,174,1,True,True,Green Shell,182,57,5,Mario Kart 8 Deluxe,0.222222
3,Yoshi,Toad Brigade,Star World,Pipe Frame,Goomba,C,D,A,23,Drifter,-1,5,True,True,1-Up Mushroom,333,84,6,Super Mario Bros.,0.250000
4,Bowser Jr.,Koopa Clan,Mushroom Kingdom,Pipe Frame,Toad,C,C,B,10,Blocker,28,2,True,True,"Red Shell, Banana Peel, Fire Flower",461,55,7,Super Mario World,0.166667


## Player-Level Summary Statistics 📊
Aggregate the data to create player-level summary statistics, such as total coins spent, total levels completed, and average ranks.

### 1. Team Performance: Aggregate team points, average rank, and total coins spent per team

In [4]:
# 1. Team Performance: Aggregate team points, average rank, and total coins spent per team

# Map letter grades to numbers for mean calculation
rank_map = {'S': 5, 'A': 4, 'B': 3, 'C': 2, 'D': 1}
df['Kart Racing Rank Num'] = df['Kart Racing Rank'].map(rank_map)

# Group by Team and aggregate the required metrics
team_summary = df.groupby('Team').agg({
    'Team Points': 'sum',
    'Kart Racing Rank Num': 'mean',
    'Coins Spent in Toad Town': 'sum'
}).reset_index()

# Round the mean Kart Racing Rank Num to 2 decimal places
team_summary['Kart Racing Rank Num'] = team_summary['Kart Racing Rank Num'].round(2)

team_summary.head()

,Team,Team Points,Kart Racing Rank Num,Coins Spent in Toad Town
0,Dino Buddies,274062,3.00,1363819
1,Green Caps,283310,3.01,1411585
2,Koopa Clan,282558,2.99,1375545
3,Mushroom Royalty,284478,3.00,1438670
4,No Team,138202,3.04,741924


In [30]:
team_summary.to_csv('1.team_performance.csv', index=False)

In [27]:
# Save team_summary to parquet
team_summary.to_parquet('../Medallion Architecture/gold/gold_team_summary.parquet', index=False)

### 2. Player Performance Ranking

In [28]:
# 2. Player Performance Ranking
player_performance = df.groupby('Player Name')['Team Points'].sum().sort_values(ascending=False)
player_performance

Player Name
Unknown Player    454698
Luigi             172753
Bowser Jr.        170904
Bowser            165782
Waluigi           165295
Mario             163519
Daisy             163110
Rosalina          162711
Toad              161178
Peach             159581
Wario             157165
Yoshi             153025
Toadette          150295
Name: Team Points, dtype: int32

In [33]:
player_performance.to_csv('2.player_performance.csv', index=True)

In [7]:
# Save player_performance to parquet
player_performance.to_frame().to_parquet('../Medallion Architecture/gold/gold_player_performance.parquet')

### 3. Power-Up Usage: Count and average of power-ups used per player

In [8]:
# 3. Power-Up Usage: Count and average of power-ups used per player and per team
powerup_player = df.groupby('Player Name')['Power-Ups Used'].agg(['sum', 'mean']).reset_index()
powerup_player['mean'] = powerup_player['mean'].round(2)

powerup_player.head()


,Player Name,sum,mean
0,Bowser,56889,17.41
1,Bowser Jr.,56800,17.53
2,Daisy,57556,17.47
3,Luigi,56875,17.40
4,Mario,54703,17.20


In [32]:
powerup_player.to_csv('3.powerup_usage_per_player.csv', index=False)

In [9]:
powerup_player.to_parquet('../Medallion Architecture/gold/gold_powerup_player.parquet', index=False)

### 4. World Difficulty (by lives lost)

In [10]:
# 4. World Difficulty (by lives lost)
world_difficulty = df.groupby('World')['Lives Lost'].mean().sort_values(ascending=False)
world_difficulty
world_difficulty = world_difficulty.round(2)
world_difficulty

World
Donut Plains        2.53
Mushroom Kingdom    2.53
Toad Town           2.51
Yoshi's Island      2.50
Star World          2.50
Koopa Kingdom       2.48
Name: Lives Lost, dtype: float64

In [11]:
world_difficulty.to_frame().to_parquet('../Medallion Architecture/gold/gold_world_difficulty.parquet', index=False)

In [34]:
world_difficulty.to_csv('4.world_difficulty.csv', index=True)

### 5. Top Player per Team by Team Points

In [12]:
# 11. Top Player per Team by Team Points
top_player_per_team = df.loc[df.groupby('Team')['Team Points'].idxmax()][['Team', 'Player Name', 'Team Points']].reset_index(drop=True)
top_player_per_team


,Team,Player Name,Team Points
0,Dino Buddies,Waluigi,200
1,Green Caps,Mario,200
2,Koopa Clan,Bowser Jr.,200
3,Mushroom Royalty,Bowser,200
4,No Team,Waluigi,200
5,Red Caps,Waluigi,200
6,Sarasaland Stars,Waluigi,200
7,Toad Brigade,Peach,200
8,Tricksters,Waluigi,200


In [13]:
top_player_per_team.to_parquet('../Medallion Architecture/gold/gold_top_player_per_team.parquet', index=False)

In [37]:
top_player_per_team.to_csv('5.top_player_per_team.csv', index=False)

### 6. Vehicle Type Popularity: Count of each vehicle type used

In [14]:
# 4. Vehicle Type Popularity: Count of each vehicle type used
vehicle_counts = df['Vehicle Type'].value_counts().reset_index()
vehicle_counts.columns = ['Vehicle Type', 'Count']
vehicle_counts

,Vehicle Type,Count
0,Scooter,6916
1,Standard Kart,6914
2,Pipe Frame,6868
3,Biddybuggy,6863
4,Mach 8,6839
5,Comet Bike,6828
6,Circuit Special,6772


In [15]:
# Save vehicle_counts to parquet
vehicle_counts.to_parquet('../Medallion Architecture/gold/gold_vehicle_counts.parquet', index=False)

In [36]:
vehicle_counts.to_csv('6.vehicle_type_popularity.csv', index=False)

### 7. High Risk Players (most lives lost)

In [16]:
# 7. High Risk Players (most lives lost)
risk_assessment = df.groupby('Player Name')['Lives Lost'].sum().sort_values(ascending=False)
risk_assessment


Player Name
Unknown Player    22667
Yoshi              8367
Waluigi            8349
Toadette           8338
Daisy              8214
Peach              8166
Bowser Jr.         8160
Luigi              8103
Rosalina           8092
Toad               8085
Bowser             8072
Mario              8009
Wario              7754
Name: Lives Lost, dtype: int32

In [38]:
risk_assessment.to_frame().to_csv('7.risk_assessment.csv', index=True)

In [17]:
risk_assessment.to_frame().to_parquet('../Medallion Architecture/gold/gold_risk_assessment.parquet')

### 8. World/Level Completion: Average and total levels completed per world

In [18]:
# 5. World/Level Completion: Average and total levels completed per world
world_completion = df.groupby('World')['Levels Completed'].agg(['sum', 'mean']).reset_index()
world_completion['mean'] = world_completion['mean'].round(2)
world_completion

,World,sum,mean
0,Donut Plains,507269,62.73
1,Koopa Kingdom,503194,62.69
2,Mushroom Kingdom,491277,61.90
3,Star World,496809,62.25
4,Toad Town,501699,62.50
5,Yoshi's Island,497014,62.57


In [39]:
world_completion.to_csv('8.world_completion.csv', index=False)

In [19]:
world_completion.to_parquet('../Medallion Architecture/gold/gold_world_completion.parquet', index=False)

### 9. Error/Hit Analysis: Total and average times hit by enemies per team

In [20]:
hits_team = df.groupby('Team')['Times Hit by Enemies'].agg(['sum', 'mean']).reset_index()
hits_team['mean'] = hits_team['mean'].round(2)
hits_team.head()

,Team,sum,mean
0,Dino Buddies,27417,4.98
1,Green Caps,27943,4.97
2,Koopa Clan,28023,5.04
3,Mushroom Royalty,28368,4.97
4,No Team,14965,5.06


In [21]:
hits_team.to_parquet('../Medallion Architecture/gold/gold_hits_team.parquet', index=False)

In [40]:
hits_team.to_csv('9.hits_per_team.csv', index=False)

### 10. Spending Analysis by Team

In [22]:
# 10. Spending Analysis
spending_analysis = df.groupby('Team')['Coins Spent in Toad Town'].sum().sort_values(ascending=False)
spending_analysis


Team
Mushroom Royalty    1438670
Sarasaland Stars    1426623
Green Caps          1411585
Red Caps            1409952
Toad Brigade        1395065
Tricksters          1394614
Koopa Clan          1375545
Dino Buddies        1363819
No Team              741924
Name: Coins Spent in Toad Town, dtype: int32

In [23]:
spending_analysis.to_frame().to_parquet('../Medallion Architecture/gold/gold_spending_analysis.parquet', index=False)

In [41]:
spending_analysis.to_csv('10.spending_analysis.csv', index=True)